# Calculo de las metricas de un json

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')

### Lectura del documento

In [ ]:
# Ruta del archivo con los datos obtenidos por el "Tracking system"
file_path = './RESULT.json'

In [ ]:
# Abrir el archivo json y cargarlo en la variable data
with open(file_path) as f:
    data = json.load(f)

# Se ordena la información del documento en base el timestamp
sorted_data = sorted(data, key=lambda x: x['timestamp'])

In [ ]:
# Numero total de eventos
num_events = len(sorted_data)
# Indice de eventos
index = 0
positions = []
sesion = 0
test_list = []

In [ ]:
def parseEvent(event):
    global index_game
    global test_list
    global sesion
    global level
    global game_list
    if event['typeEvent'] == "LoginEvent":
        print("\nInicio de sesion: ", sesion)
    elif event['typeEvent'] == "LogoutEvent":
        print("\nFin de sesion: ", sesion)
        sesion += 1
    # Empieza el test
    elif event['typeEvent'] == "StartTestEvent":
       
    # Acaba la partida
    elif event['typeEvent'] == "EndTestEvent":
        
    elif event['typeEvent'] == "BotPositionEvent":
        test_list.append(event)
    return  True

In [ ]:
# Recorro todos los eventos generados
while index < num_events:
    currentEvent = sorted_data[index]
    consumeEvent = parseEvent(currentEvent)
    index += 1

In [ ]:
dfTest = pd.DataFrame(test_list)
dfTest.head()

In [ ]:
# Definir las dimensiones del mapa
fig,ax = plt.subplots(figsize=(200,200)) #Deberia ser un parametro del test

# Guardar la imagen del mapa
img = plt.imread("images/map1.png")
ax.imshow(img)

In [ ]:
# Dibujar el mapa de calor
dfTest.plot.hexbin(fig = fig, ax = ax,x="posX", y="posZ", reduce_C_function=sum, gridsize=(200,200), extent=[0,750,0,750], alpha=0.5, cmap='Reds')
ax.set_xticks(range(0, 750, 100))
ax.set_yticks(range(0, 750, 100))
fig.savefig('heatmap.png')